In [ ]:
%matplotlib inline

import sys 
import os 
import copy 
import numpy as np

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

In [ ]:
emb_names = ['all-paras-lower-100-skip-minoccur-50', 'aus-100-cbow-min-50-k30-n100', 'flattened-wiki-news-300d-1M']
models = ['mnzexp', 'maxexp', 'centroid', 'tlm']
folders = ['mnz', 'max', 'centroid', 'tlm']

def load_exp_dfs(qrel_path, results_path, run_format, rel_level, emb_names, models, model_dirs, per_query=False, filtered=None):
    dfs = []
    for i, e in enumerate(emb_names):
        temp = []
        for j, m in enumerate(models):
            ltemp = []
            for k in range(1, 21): 
                    ltemp.append(to_trec_df(qrel_path, os.path.join(results_path, model_dirs[j], run_format.format(e, m, k)), rel_level, per_query, filtered))
            temp.append(ltemp)
        dfs.append(temp)

    return dfs 

tlm_dfs = load_exp_dfs(config.AUS_QREL_PATH, os.path.join(BASE_DIR, 'expansion'), 'flattened-{0}-{1}-dir-mu-2400.00-neighbours-{2}.run', '1', emb_names, models, folders)

In [ ]:
base_res = load_1d_dfs(['filtered-phrasestop'], [config.AUS_QREL_PATH], os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', [AUS_REL_LEVEL], 1050.0, 1050.0, 50.0)[0][0]

In [ ]:
# def plot_tune_1d_comp(names, metric_names, dfs, start, end, increment, legend_x: float=0.96, legend_y: float=0.46): 

#     r = int(len(metric_names)/2)
#     c = r
#     if c == r: 
#         r-=1
#     if len(metric_names) % 2 != 0:
#         c += 1
#     fig, axs = plt.subplots(r, c)
#     fig.set_size_inches(16, 6)
#     x = np.arange(start, end+increment, increment)
#     cnt = 0 
#     row = 0
#     print(len(x))
#     for m in metric_names:
#             for df in dfs: 
#                 axs[row, cnt].plot(x, [y[m] for y in df])

#             axs[row, cnt].set_ylabel(metric_names[m],fontsize=18)

#             axs[row, cnt].tick_params(labelsize=12)
#             axs[row, cnt].yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
#             cnt += 1 
#             if cnt >= c: 
#                 cnt = 0 
#                 row += 1 
    
#     if len(metric_names) % 2 != 0: 
#         fig.delaxes(axs[row, -1])

#     fig.legend(names, bbox_to_anchor=[legend_x, legend_y], frameon=True, ncol=2, prop={"size": 15}).get_frame().set_edgecolor('black')
        
#     fig.tight_layout()
#     return fig

In [ ]:
comp_fig = plot_tune_1d_comp(['base']+models, RERANK_METRICS, [[base_res for x in range(20)]] + tlm_dfs[1], 1, 20, 1, ylims=RERANK_YLIMS, styles=['--'])

In [ ]:
# comp_fig.savefig('figures/ausnl-embedding-expansion.pdf')

In [ ]:
comp_fig = plot_tune_1d_comp(['base']+models, RERANK_METRICS, [[base_res for x in range(20)]] + tlm_dfs[0], 1, 20, 1, legend_y=0.25)

In [ ]:
baseline_res = load_1d_dfs(['case-topics'], [config.AUS_QREL_PATH], os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), '{0}-filtered-phrasestop-unigram_dir_mu_{1:.2f}.run', [AUS_REL_LEVEL], 1050, 1050, 50)

In [ ]:
def plot_tune_1d(index_names, metric_names, df, start, end, increment): 
    fig, axs = plt.subplots(len(metric_names), len(index_names), sharex=True)
    fig.set_size_inches(16, 10)
    for i in range(len(index_names)):
        cnt = 0
        for m in metric_names:
            y = [y[m] for y in df[i]]
            axs[cnt, i].plot(np.arange(start, end, increment), y)

            if i == 0:
                axs[cnt, i].set_ylabel(metric_names[m],fontsize=18)

            axs[cnt, i].tick_params(labelsize=12)
            axs[cnt, i].yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
            cnt += 1
                
        axs[cnt-1, i].set_xlabel(index_names[i], fontsize=20)
        
    fig.tight_layout()
    return fig 


tlm_fig = plot_tune_1d(models, RERANK_METRICS, tlm_dfs[0], 1, 21, 1)